In [3]:
# InO to OOO
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions:u']
            cyc = row['cpu-cycles:u']
            ipc = inst/cyc
            features.append([
                row['instructions:u'],
                row['cpu-cycles:u'],
                row['armv8_pmuv3/stall_backend/u'],
                row['armv8_pmuv3/stall_frontend/u'],
                row['armv8_pmuv3/br_mis_pred/u'],
                row['armv8_pmuv3/br_pred/u'],
                row['armv8_pmuv3/br_retired/u'],
                row['armv8_pmuv3/inst_spec/u'],
                row['armv8_pmuv3/inst_retired/u'],
                row['armv8_pmuv3/l1d_cache/u'],
                row['armv8_pmuv3/l1d_cache_refill/u'],
                row['armv8_pmuv3/l1d_cache_wb/u'],
                row['armv8_pmuv3/l1d_tlb/u'],
                row['armv8_pmuv3/l1d_tlb_refill/u'],
                row['armv8_pmuv3/l1i_cache/u'],
                row['armv8_pmuv3/l1i_cache_refill/u'],
                row['armv8_pmuv3/l1i_tlb/u'],
                row['armv8_pmuv3/l1i_tlb_refill/u'],
                row['armv8_pmuv3/l2d_cache/u'],
                row['armv8_pmuv3/l2d_cache_allocate/u'],
                row['armv8_pmuv3/l2d_cache_refill/u'],
                row['armv8_pmuv3/l2d_cache_wb/u'],
                row['armv8_pmuv3/l2d_tlb/u'],
                row['armv8_pmuv3/l2d_tlb_refill/u'],
                row['armv8_pmuv3/l3d_cache/u'],
                row['armv8_pmuv3/l3d_cache_allocate/u'],
                row['armv8_pmuv3/l3d_cache_refill/u'],
                row['armv8_pmuv3/l3d_cache_wb/u'],
                row['armv8_pmuv3/mem_access/u'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions:u'] / row['cpu-cycles:u'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_554',
]

chunks = [200, 400, 600, 800, 1000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions:u',
            'cpu-cycles:u',
            'armv8_pmuv3/stall_backend/u',
            'armv8_pmuv3/stall_frontend/u',
            'armv8_pmuv3/br_mis_pred/u',
            'armv8_pmuv3/br_pred/u',
            'armv8_pmuv3/br_retired/u',
            'armv8_pmuv3/inst_spec/u',
            'armv8_pmuv3/inst_retired/u',
            'armv8_pmuv3/l1d_cache/u',
            'armv8_pmuv3/l1d_cache_refill/u',
            'armv8_pmuv3/l1d_cache_wb/u',
            'armv8_pmuv3/l1d_tlb/u',
            'armv8_pmuv3/l1d_tlb_refill/u',
            'armv8_pmuv3/l1i_cache/u',
            'armv8_pmuv3/l1i_cache_refill/u',
            'armv8_pmuv3/l1i_tlb/u',
            'armv8_pmuv3/l1i_tlb_refill/u',
            'armv8_pmuv3/l2d_cache/u',
            'armv8_pmuv3/l2d_cache_allocate/u',
            'armv8_pmuv3/l2d_cache_refill/u',
            'armv8_pmuv3/l2d_cache_wb/u',
            'armv8_pmuv3/l2d_tlb/u',
            'armv8_pmuv3/l2d_tlb_refill/u',
            'armv8_pmuv3/l3d_cache/u',
            'armv8_pmuv3/l3d_cache_allocate/u',
            'armv8_pmuv3/l3d_cache_refill/u',
            'armv8_pmuv3/l3d_cache_wb/u',
            'armv8_pmuv3/mem_access/u',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 200
Training R²: 0.9913012473746192
Testing R²: 0.9868485686886534
Training Mape: 0.02292627608924239
Testing Mape: 0.027968745470932373
chunk_size: 400
Training R²: 0.944281744575588
Testing R²: 0.9066774166282804
Training Mape: 0.051784728585945794
Testing Mape: 0.06414485839881504
chunk_size: 600
Training R²: 0.9108954632426258
Testing R²: 0.8885947201584979
Training Mape: 0.06771206036159785
Testing Mape: 0.07589914596517532
chunk_size: 800
Training R²: 0.8869809718846426
Testing R²: 0.8551448579306697
Training Mape: 0.07784398939193223
Testing Mape: 0.08233339226401971
chunk_size: 1000
Training R²: 0.8593125170244575
Testing R²: 0.842426579848459
Training Mape: 0.08758830008685584
Testing Mape: 0.09221700890884169


In [5]:
# InO to desktop
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions:u']
            cyc = row['cpu-cycles:u']
            ipc = inst/cyc
            features.append([
                row['instructions:u'],
                row['cpu-cycles:u'],
                row['armv8_pmuv3/stall_backend/u'],
                row['armv8_pmuv3/stall_frontend/u'],
                row['armv8_pmuv3/br_mis_pred/u'],
                row['armv8_pmuv3/br_pred/u'],
                row['armv8_pmuv3/br_retired/u'],
                row['armv8_pmuv3/inst_spec/u'],
                row['armv8_pmuv3/inst_retired/u'],
                row['armv8_pmuv3/l1d_cache/u'],
                row['armv8_pmuv3/l1d_cache_refill/u'],
                row['armv8_pmuv3/l1d_cache_wb/u'],
                row['armv8_pmuv3/l1d_tlb/u'],
                row['armv8_pmuv3/l1d_tlb_refill/u'],
                row['armv8_pmuv3/l1i_cache/u'],
                row['armv8_pmuv3/l1i_cache_refill/u'],
                row['armv8_pmuv3/l1i_tlb/u'],
                row['armv8_pmuv3/l1i_tlb_refill/u'],
                row['armv8_pmuv3/l2d_cache/u'],
                row['armv8_pmuv3/l2d_cache_allocate/u'],
                row['armv8_pmuv3/l2d_cache_refill/u'],
                row['armv8_pmuv3/l2d_cache_wb/u'],
                row['armv8_pmuv3/l2d_tlb/u'],
                row['armv8_pmuv3/l2d_tlb_refill/u'],
                row['armv8_pmuv3/l3d_cache/u'],
                row['armv8_pmuv3/l3d_cache_allocate/u'],
                row['armv8_pmuv3/l3d_cache_refill/u'],
                row['armv8_pmuv3/l3d_cache_wb/u'],
                row['armv8_pmuv3/mem_access/u'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions'] / row['cpu-cycles'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_554',
]

chunks = [200, 400, 600, 800, 1000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions:u',
            'cpu-cycles:u',
            'armv8_pmuv3/stall_backend/u',
            'armv8_pmuv3/stall_frontend/u',
            'armv8_pmuv3/br_mis_pred/u',
            'armv8_pmuv3/br_pred/u',
            'armv8_pmuv3/br_retired/u',
            'armv8_pmuv3/inst_spec/u',
            'armv8_pmuv3/inst_retired/u',
            'armv8_pmuv3/l1d_cache/u',
            'armv8_pmuv3/l1d_cache_refill/u',
            'armv8_pmuv3/l1d_cache_wb/u',
            'armv8_pmuv3/l1d_tlb/u',
            'armv8_pmuv3/l1d_tlb_refill/u',
            'armv8_pmuv3/l1i_cache/u',
            'armv8_pmuv3/l1i_cache_refill/u',
            'armv8_pmuv3/l1i_tlb/u',
            'armv8_pmuv3/l1i_tlb_refill/u',
            'armv8_pmuv3/l2d_cache/u',
            'armv8_pmuv3/l2d_cache_allocate/u',
            'armv8_pmuv3/l2d_cache_refill/u',
            'armv8_pmuv3/l2d_cache_wb/u',
            'armv8_pmuv3/l2d_tlb/u',
            'armv8_pmuv3/l2d_tlb_refill/u',
            'armv8_pmuv3/l3d_cache/u',
            'armv8_pmuv3/l3d_cache_allocate/u',
            'armv8_pmuv3/l3d_cache_refill/u',
            'armv8_pmuv3/l3d_cache_wb/u',
            'armv8_pmuv3/mem_access/u',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 200
Training R²: 0.986534096874625
Testing R²: 0.9751675441128445
Training Mape: 0.02871735575715219
Testing Mape: 0.03226719599562379
chunk_size: 400
Training R²: 0.9533168803461105
Testing R²: 0.9383388190554369
Training Mape: 0.048416003393265426
Testing Mape: 0.05639164320418491
chunk_size: 600
Training R²: 0.9188472299639803
Testing R²: 0.904414968398469
Training Mape: 0.06871137703505756
Testing Mape: 0.07315157701695552
chunk_size: 800
Training R²: 0.9046297722771992
Testing R²: 0.8844572150812117
Training Mape: 0.07842162915179596
Testing Mape: 0.0839158917160562
chunk_size: 1000
Training R²: 0.8900405539508415
Testing R²: 0.8746398000634155
Training Mape: 0.08536399910834018
Testing Mape: 0.08947419764153604


In [6]:
# InO to server
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions:u']
            cyc = row['cpu-cycles:u']
            ipc = inst/cyc
            features.append([
                row['instructions:u'],
                row['cpu-cycles:u'],
                row['armv8_pmuv3/stall_backend/u'],
                row['armv8_pmuv3/stall_frontend/u'],
                row['armv8_pmuv3/br_mis_pred/u'],
                row['armv8_pmuv3/br_pred/u'],
                row['armv8_pmuv3/br_retired/u'],
                row['armv8_pmuv3/inst_spec/u'],
                row['armv8_pmuv3/inst_retired/u'],
                row['armv8_pmuv3/l1d_cache/u'],
                row['armv8_pmuv3/l1d_cache_refill/u'],
                row['armv8_pmuv3/l1d_cache_wb/u'],
                row['armv8_pmuv3/l1d_tlb/u'],
                row['armv8_pmuv3/l1d_tlb_refill/u'],
                row['armv8_pmuv3/l1i_cache/u'],
                row['armv8_pmuv3/l1i_cache_refill/u'],
                row['armv8_pmuv3/l1i_tlb/u'],
                row['armv8_pmuv3/l1i_tlb_refill/u'],
                row['armv8_pmuv3/l2d_cache/u'],
                row['armv8_pmuv3/l2d_cache_allocate/u'],
                row['armv8_pmuv3/l2d_cache_refill/u'],
                row['armv8_pmuv3/l2d_cache_wb/u'],
                row['armv8_pmuv3/l2d_tlb/u'],
                row['armv8_pmuv3/l2d_tlb_refill/u'],
                row['armv8_pmuv3/l3d_cache/u'],
                row['armv8_pmuv3/l3d_cache_allocate/u'],
                row['armv8_pmuv3/l3d_cache_refill/u'],
                row['armv8_pmuv3/l3d_cache_wb/u'],
                row['armv8_pmuv3/mem_access/u'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions'] / row['cpu-cycles'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_554',
]

chunks = [200, 400, 600, 800, 1000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions:u',
            'cpu-cycles:u',
            'armv8_pmuv3/stall_backend/u',
            'armv8_pmuv3/stall_frontend/u',
            'armv8_pmuv3/br_mis_pred/u',
            'armv8_pmuv3/br_pred/u',
            'armv8_pmuv3/br_retired/u',
            'armv8_pmuv3/inst_spec/u',
            'armv8_pmuv3/inst_retired/u',
            'armv8_pmuv3/l1d_cache/u',
            'armv8_pmuv3/l1d_cache_refill/u',
            'armv8_pmuv3/l1d_cache_wb/u',
            'armv8_pmuv3/l1d_tlb/u',
            'armv8_pmuv3/l1d_tlb_refill/u',
            'armv8_pmuv3/l1i_cache/u',
            'armv8_pmuv3/l1i_cache_refill/u',
            'armv8_pmuv3/l1i_tlb/u',
            'armv8_pmuv3/l1i_tlb_refill/u',
            'armv8_pmuv3/l2d_cache/u',
            'armv8_pmuv3/l2d_cache_allocate/u',
            'armv8_pmuv3/l2d_cache_refill/u',
            'armv8_pmuv3/l2d_cache_wb/u',
            'armv8_pmuv3/l2d_tlb/u',
            'armv8_pmuv3/l2d_tlb_refill/u',
            'armv8_pmuv3/l3d_cache/u',
            'armv8_pmuv3/l3d_cache_allocate/u',
            'armv8_pmuv3/l3d_cache_refill/u',
            'armv8_pmuv3/l3d_cache_wb/u',
            'armv8_pmuv3/mem_access/u',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 200
Training R²: 0.9803388430909404
Testing R²: 0.971101204684222
Training Mape: 0.025276912655404393
Testing Mape: 0.030528780231993975
chunk_size: 400
Training R²: 0.9197406488297424
Testing R²: 0.8762491801593958
Training Mape: 0.04844743139477376
Testing Mape: 0.05674320404097222
chunk_size: 600
Training R²: 0.8772356639526557
Testing R²: 0.8650551396893699
Training Mape: 0.06214332224666701
Testing Mape: 0.06639898467982713
chunk_size: 800
Training R²: 0.8512823619291654
Testing R²: 0.8281825264385427
Training Mape: 0.07047904567398386
Testing Mape: 0.07505374051805816
chunk_size: 1000
Training R²: 0.821395408704361
Testing R²: 0.8102495955616588
Training Mape: 0.08108366340515416
Testing Mape: 0.08348434479697467


In [7]:
# OOO to InO
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions:u']
            cyc = row['cpu-cycles:u']
            ipc = inst/cyc
            features.append([
                row['instructions:u'],
                row['cpu-cycles:u'],
                row['armv8_pmuv3/stall_backend/u'],
                row['armv8_pmuv3/stall_frontend/u'],
                row['armv8_pmuv3/br_mis_pred/u'],
                row['armv8_pmuv3/br_pred/u'],
                row['armv8_pmuv3/br_retired/u'],
                row['armv8_pmuv3/inst_spec/u'],
                row['armv8_pmuv3/inst_retired/u'],
                row['armv8_pmuv3/l1d_cache/u'],
                row['armv8_pmuv3/l1d_cache_refill/u'],
                row['armv8_pmuv3/l1d_cache_wb/u'],
                row['armv8_pmuv3/l1d_tlb/u'],
                row['armv8_pmuv3/l1d_tlb_refill/u'],
                row['armv8_pmuv3/l1i_cache/u'],
                row['armv8_pmuv3/l1i_cache_refill/u'],
                row['armv8_pmuv3/l1i_tlb/u'],
                row['armv8_pmuv3/l1i_tlb_refill/u'],
                row['armv8_pmuv3/l2d_cache/u'],
                row['armv8_pmuv3/l2d_cache_allocate/u'],
                row['armv8_pmuv3/l2d_cache_refill/u'],
                row['armv8_pmuv3/l2d_cache_wb/u'],
                row['armv8_pmuv3/l2d_tlb/u'],
                row['armv8_pmuv3/l2d_tlb_refill/u'],
                row['armv8_pmuv3/l3d_cache/u'],
                row['armv8_pmuv3/l3d_cache_allocate/u'],
                row['armv8_pmuv3/l3d_cache_refill/u'],
                row['armv8_pmuv3/l3d_cache_wb/u'],
                row['armv8_pmuv3/mem_access/u'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions:u'] / row['cpu-cycles:u'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_554',
]

chunks = [200, 400, 600, 800, 1000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions:u',
            'cpu-cycles:u',
            'armv8_pmuv3/stall_backend/u',
            'armv8_pmuv3/stall_frontend/u',
            'armv8_pmuv3/br_mis_pred/u',
            'armv8_pmuv3/br_pred/u',
            'armv8_pmuv3/br_retired/u',
            'armv8_pmuv3/inst_spec/u',
            'armv8_pmuv3/inst_retired/u',
            'armv8_pmuv3/l1d_cache/u',
            'armv8_pmuv3/l1d_cache_refill/u',
            'armv8_pmuv3/l1d_cache_wb/u',
            'armv8_pmuv3/l1d_tlb/u',
            'armv8_pmuv3/l1d_tlb_refill/u',
            'armv8_pmuv3/l1i_cache/u',
            'armv8_pmuv3/l1i_cache_refill/u',
            'armv8_pmuv3/l1i_tlb/u',
            'armv8_pmuv3/l1i_tlb_refill/u',
            'armv8_pmuv3/l2d_cache/u',
            'armv8_pmuv3/l2d_cache_allocate/u',
            'armv8_pmuv3/l2d_cache_refill/u',
            'armv8_pmuv3/l2d_cache_wb/u',
            'armv8_pmuv3/l2d_tlb/u',
            'armv8_pmuv3/l2d_tlb_refill/u',
            'armv8_pmuv3/l3d_cache/u',
            'armv8_pmuv3/l3d_cache_allocate/u',
            'armv8_pmuv3/l3d_cache_refill/u',
            'armv8_pmuv3/l3d_cache_wb/u',
            'armv8_pmuv3/mem_access/u',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 200
Training R²: 0.9931870904190125
Testing R²: 0.9883070439318633
Training Mape: 0.03286739042561342
Testing Mape: 0.03993141906018072
chunk_size: 400
Training R²: 0.9390921716974631
Testing R²: 0.8830574276275086
Training Mape: 0.08338740085616547
Testing Mape: 0.0998795349860269
chunk_size: 600
Training R²: 0.9030523113745973
Testing R²: 0.8673126346291912
Training Mape: 0.11833261557693703
Testing Mape: 0.12840399613983952
chunk_size: 800
Training R²: 0.8832259117020909
Testing R²: 0.8545163120247757
Training Mape: 0.13240370207968383
Testing Mape: 0.14442972044936533
chunk_size: 1000
Training R²: 0.8597513258273279
Testing R²: 0.8517555707432907
Training Mape: 0.1484880096633163
Testing Mape: 0.15133087972518366


In [8]:
# OOO to desktop
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions:u']
            cyc = row['cpu-cycles:u']
            ipc = inst/cyc
            features.append([
                row['instructions:u'],
                row['cpu-cycles:u'],
                row['armv8_pmuv3/stall_backend/u'],
                row['armv8_pmuv3/stall_frontend/u'],
                row['armv8_pmuv3/br_mis_pred/u'],
                row['armv8_pmuv3/br_pred/u'],
                row['armv8_pmuv3/br_retired/u'],
                row['armv8_pmuv3/inst_spec/u'],
                row['armv8_pmuv3/inst_retired/u'],
                row['armv8_pmuv3/l1d_cache/u'],
                row['armv8_pmuv3/l1d_cache_refill/u'],
                row['armv8_pmuv3/l1d_cache_wb/u'],
                row['armv8_pmuv3/l1d_tlb/u'],
                row['armv8_pmuv3/l1d_tlb_refill/u'],
                row['armv8_pmuv3/l1i_cache/u'],
                row['armv8_pmuv3/l1i_cache_refill/u'],
                row['armv8_pmuv3/l1i_tlb/u'],
                row['armv8_pmuv3/l1i_tlb_refill/u'],
                row['armv8_pmuv3/l2d_cache/u'],
                row['armv8_pmuv3/l2d_cache_allocate/u'],
                row['armv8_pmuv3/l2d_cache_refill/u'],
                row['armv8_pmuv3/l2d_cache_wb/u'],
                row['armv8_pmuv3/l2d_tlb/u'],
                row['armv8_pmuv3/l2d_tlb_refill/u'],
                row['armv8_pmuv3/l3d_cache/u'],
                row['armv8_pmuv3/l3d_cache_allocate/u'],
                row['armv8_pmuv3/l3d_cache_refill/u'],
                row['armv8_pmuv3/l3d_cache_wb/u'],
                row['armv8_pmuv3/mem_access/u'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions'] / row['cpu-cycles'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_554',
]

chunks = [200, 400, 600, 800, 1000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions:u',
            'cpu-cycles:u',
            'armv8_pmuv3/stall_backend/u',
            'armv8_pmuv3/stall_frontend/u',
            'armv8_pmuv3/br_mis_pred/u',
            'armv8_pmuv3/br_pred/u',
            'armv8_pmuv3/br_retired/u',
            'armv8_pmuv3/inst_spec/u',
            'armv8_pmuv3/inst_retired/u',
            'armv8_pmuv3/l1d_cache/u',
            'armv8_pmuv3/l1d_cache_refill/u',
            'armv8_pmuv3/l1d_cache_wb/u',
            'armv8_pmuv3/l1d_tlb/u',
            'armv8_pmuv3/l1d_tlb_refill/u',
            'armv8_pmuv3/l1i_cache/u',
            'armv8_pmuv3/l1i_cache_refill/u',
            'armv8_pmuv3/l1i_tlb/u',
            'armv8_pmuv3/l1i_tlb_refill/u',
            'armv8_pmuv3/l2d_cache/u',
            'armv8_pmuv3/l2d_cache_allocate/u',
            'armv8_pmuv3/l2d_cache_refill/u',
            'armv8_pmuv3/l2d_cache_wb/u',
            'armv8_pmuv3/l2d_tlb/u',
            'armv8_pmuv3/l2d_tlb_refill/u',
            'armv8_pmuv3/l3d_cache/u',
            'armv8_pmuv3/l3d_cache_allocate/u',
            'armv8_pmuv3/l3d_cache_refill/u',
            'armv8_pmuv3/l3d_cache_wb/u',
            'armv8_pmuv3/mem_access/u',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 200
Training R²: 0.9820593782209812
Testing R²: 0.9697673830561647
Training Mape: 0.03176374484046091
Testing Mape: 0.03704360520029021
chunk_size: 400
Training R²: 0.9441580001325022
Testing R²: 0.9069341055401812
Training Mape: 0.05629138720818884
Testing Mape: 0.0655394411058626
chunk_size: 600
Training R²: 0.9127005181263393
Testing R²: 0.9064500886118066
Training Mape: 0.07086157638407796
Testing Mape: 0.07195454128044998
chunk_size: 800
Training R²: 0.8961783891793188
Testing R²: 0.8864692500308409
Training Mape: 0.08062976503935256
Testing Mape: 0.08550211552713717
chunk_size: 1000
Training R²: 0.8774881117267449
Testing R²: 0.8658562769257003
Training Mape: 0.0900138619331687
Testing Mape: 0.0940298462610939


In [9]:
# OOO to server
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions:u']
            cyc = row['cpu-cycles:u']
            ipc = inst/cyc
            features.append([
                row['instructions:u'],
                row['cpu-cycles:u'],
                row['armv8_pmuv3/stall_backend/u'],
                row['armv8_pmuv3/stall_frontend/u'],
                row['armv8_pmuv3/br_mis_pred/u'],
                row['armv8_pmuv3/br_pred/u'],
                row['armv8_pmuv3/br_retired/u'],
                row['armv8_pmuv3/inst_spec/u'],
                row['armv8_pmuv3/inst_retired/u'],
                row['armv8_pmuv3/l1d_cache/u'],
                row['armv8_pmuv3/l1d_cache_refill/u'],
                row['armv8_pmuv3/l1d_cache_wb/u'],
                row['armv8_pmuv3/l1d_tlb/u'],
                row['armv8_pmuv3/l1d_tlb_refill/u'],
                row['armv8_pmuv3/l1i_cache/u'],
                row['armv8_pmuv3/l1i_cache_refill/u'],
                row['armv8_pmuv3/l1i_tlb/u'],
                row['armv8_pmuv3/l1i_tlb_refill/u'],
                row['armv8_pmuv3/l2d_cache/u'],
                row['armv8_pmuv3/l2d_cache_allocate/u'],
                row['armv8_pmuv3/l2d_cache_refill/u'],
                row['armv8_pmuv3/l2d_cache_wb/u'],
                row['armv8_pmuv3/l2d_tlb/u'],
                row['armv8_pmuv3/l2d_tlb_refill/u'],
                row['armv8_pmuv3/l3d_cache/u'],
                row['armv8_pmuv3/l3d_cache_allocate/u'],
                row['armv8_pmuv3/l3d_cache_refill/u'],
                row['armv8_pmuv3/l3d_cache_wb/u'],
                row['armv8_pmuv3/mem_access/u'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions'] / row['cpu-cycles'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_554',
]

chunks = [200, 400, 600, 800, 1000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions:u',
            'cpu-cycles:u',
            'armv8_pmuv3/stall_backend/u',
            'armv8_pmuv3/stall_frontend/u',
            'armv8_pmuv3/br_mis_pred/u',
            'armv8_pmuv3/br_pred/u',
            'armv8_pmuv3/br_retired/u',
            'armv8_pmuv3/inst_spec/u',
            'armv8_pmuv3/inst_retired/u',
            'armv8_pmuv3/l1d_cache/u',
            'armv8_pmuv3/l1d_cache_refill/u',
            'armv8_pmuv3/l1d_cache_wb/u',
            'armv8_pmuv3/l1d_tlb/u',
            'armv8_pmuv3/l1d_tlb_refill/u',
            'armv8_pmuv3/l1i_cache/u',
            'armv8_pmuv3/l1i_cache_refill/u',
            'armv8_pmuv3/l1i_tlb/u',
            'armv8_pmuv3/l1i_tlb_refill/u',
            'armv8_pmuv3/l2d_cache/u',
            'armv8_pmuv3/l2d_cache_allocate/u',
            'armv8_pmuv3/l2d_cache_refill/u',
            'armv8_pmuv3/l2d_cache_wb/u',
            'armv8_pmuv3/l2d_tlb/u',
            'armv8_pmuv3/l2d_tlb_refill/u',
            'armv8_pmuv3/l3d_cache/u',
            'armv8_pmuv3/l3d_cache_allocate/u',
            'armv8_pmuv3/l3d_cache_refill/u',
            'armv8_pmuv3/l3d_cache_wb/u',
            'armv8_pmuv3/mem_access/u',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 200
Training R²: 0.9853871464321969
Testing R²: 0.9773144898523457
Training Mape: 0.020944644048750863
Testing Mape: 0.024590049071805172
chunk_size: 400
Training R²: 0.9479453134872452
Testing R²: 0.9279484104650808
Training Mape: 0.03840870721561051
Testing Mape: 0.04271688875206402
chunk_size: 600
Training R²: 0.925361906678976
Testing R²: 0.910593667925253
Training Mape: 0.04751269636096779
Testing Mape: 0.05111975115793863
chunk_size: 800
Training R²: 0.907749458254958
Testing R²: 0.8830923859081198
Training Mape: 0.05419376603742329
Testing Mape: 0.059783182364563515
chunk_size: 1000
Training R²: 0.8842327753521003
Testing R²: 0.872973592736151
Training Mape: 0.061303171378770324
Testing Mape: 0.0651861443540396


In [10]:
# desktop to server
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions']
            cyc = row['cpu-cycles']
            ipc = inst/cyc
            features.append([
                row['instructions'],
                row['cpu-cycles'],
                row['br_pred'],
                row['br_mis_pred'],
                row['l1d_cache_rd'],
                row['l1d_cache_wr'],
                row['l1d_cache'],
                row['l1i_cache'],
                row['l1i_cache_refill'],
                row['context-switches'],
                row['l2d_cache_rd'],
                row['l2d_cache_wr'],
                row['l2d_cache'],
                row['l1d_tlb'],
                row['l1d_tlb_refill_rd'],
                row['l1d_tlb_refill_wr'],
                row['dtlb_walk'],
                row['itlb_walk'],
                row['page-faults'],
                row['l2d_tlb_access'],
                row['l2i_tlb_access'],
                row['l1i_tlb_refill'],
                row['iTLB-loads'],
                row['iTLB-load-misses'],
                row['branch-loads'],
                row['dTLB-loads'],
                row['dTLB-load-misses'],
                row['branch-load-misses'],
                row['vfp_spec'],
                row['inst_spec'],
                row['ase_spec'],
                row['bx_stall'],
                row['decode_stall'],
                row['dispatch_stall'],
                row['fx_stall'],
                row['ixa_stall'],
                row['ixb_stall'],
                row['lx_stall'],
                row['sx_stall'],
                row['bus_access'],
                row['mem_access'],
                row['mem_access_rd'],
                row['mem_access_wr'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions'] / row['cpu-cycles'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_554',
]

chunks = [200, 400, 600, 800, 1000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions',
            'cpu-cycles',
            'br_pred',
            'br_mis_pred',
            'l1d_cache_rd',
            'l1d_cache_wr',
            'l1d_cache',
            'l1i_cache',
            'l1i_cache_refill',
            'context-switches',
            'l2d_cache_rd',
            'l2d_cache_wr',
            'l2d_cache',
            'l1d_tlb',
            'l1d_tlb_refill_rd',
            'l1d_tlb_refill_wr',
            'dtlb_walk',
            'itlb_walk',
            'page-faults',
            'l2d_tlb_access',
            'l2i_tlb_access',
            'l1i_tlb_refill',
            'iTLB-loads',
            'iTLB-load-misses',
            'branch-loads',
            'dTLB-loads',
            'dTLB-load-misses',
            'branch-load-misses',
            'vfp_spec',
            'inst_spec',
            'ase_spec',
            'bx_stall',
            'decode_stall',
            'dispatch_stall',
            'fx_stall',
            'ixa_stall',
            'ixb_stall',
            'lx_stall',
            'sx_stall',
            'bus_access',
            'mem_access',
            'mem_access_rd',
            'mem_access_wr',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 200
Training R²: 0.9814564391554962
Testing R²: 0.9718592509432137
Training Mape: 0.02397386758850154
Testing Mape: 0.028307260312597744
chunk_size: 400
Training R²: 0.9265231026017101
Testing R²: 0.897537350607259
Training Mape: 0.04520625800305858
Testing Mape: 0.05289404656888263
chunk_size: 600
Training R²: 0.896726284579211
Testing R²: 0.8767572406340154
Training Mape: 0.05649024862472983
Testing Mape: 0.061627324428061674
chunk_size: 800
Training R²: 0.8723709123944405
Testing R²: 0.8575812929543823
Training Mape: 0.06531019641505557
Testing Mape: 0.06970287529769312
chunk_size: 1000
Training R²: 0.8392050784905029
Testing R²: 0.8294692186698962
Training Mape: 0.07469225107611
Testing Mape: 0.07762525185836076


In [11]:
# desktop to InO
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions']
            cyc = row['cpu-cycles']
            ipc = inst/cyc
            features.append([
                row['instructions'],
                row['cpu-cycles'],
                row['br_pred'],
                row['br_mis_pred'],
                row['l1d_cache_rd'],
                row['l1d_cache_wr'],
                row['l1d_cache'],
                row['l1i_cache'],
                row['l1i_cache_refill'],
                row['context-switches'],
                row['l2d_cache_rd'],
                row['l2d_cache_wr'],
                row['l2d_cache'],
                row['l1d_tlb'],
                row['l1d_tlb_refill_rd'],
                row['l1d_tlb_refill_wr'],
                row['dtlb_walk'],
                row['itlb_walk'],
                row['page-faults'],
                row['l2d_tlb_access'],
                row['l2i_tlb_access'],
                row['l1i_tlb_refill'],
                row['iTLB-loads'],
                row['iTLB-load-misses'],
                row['branch-loads'],
                row['dTLB-loads'],
                row['dTLB-load-misses'],
                row['branch-load-misses'],
                row['vfp_spec'],
                row['inst_spec'],
                row['ase_spec'],
                row['bx_stall'],
                row['decode_stall'],
                row['dispatch_stall'],
                row['fx_stall'],
                row['ixa_stall'],
                row['ixb_stall'],
                row['lx_stall'],
                row['sx_stall'],
                row['bus_access'],
                row['mem_access'],
                row['mem_access_rd'],
                row['mem_access_wr'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions:u'] / row['cpu-cycles:u'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_554',
]

chunks = [200, 400, 600, 800, 1000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions',
            'cpu-cycles',
            'br_pred',
            'br_mis_pred',
            'l1d_cache_rd',
            'l1d_cache_wr',
            'l1d_cache',
            'l1i_cache',
            'l1i_cache_refill',
            'context-switches',
            'l2d_cache_rd',
            'l2d_cache_wr',
            'l2d_cache',
            'l1d_tlb',
            'l1d_tlb_refill_rd',
            'l1d_tlb_refill_wr',
            'dtlb_walk',
            'itlb_walk',
            'page-faults',
            'l2d_tlb_access',
            'l2i_tlb_access',
            'l1i_tlb_refill',
            'iTLB-loads',
            'iTLB-load-misses',
            'branch-loads',
            'dTLB-loads',
            'dTLB-load-misses',
            'branch-load-misses',
            'vfp_spec',
            'inst_spec',
            'ase_spec',
            'bx_stall',
            'decode_stall',
            'dispatch_stall',
            'fx_stall',
            'ixa_stall',
            'ixb_stall',
            'lx_stall',
            'sx_stall',
            'bus_access',
            'mem_access',
            'mem_access_rd',
            'mem_access_wr',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 200
Training R²: 0.9875323292874805
Testing R²: 0.9777710037937948
Training Mape: 0.045925862282553465
Testing Mape: 0.05183813174797467
chunk_size: 400
Training R²: 0.9646221188906265
Testing R²: 0.9350604138257119
Training Mape: 0.07421405343334593
Testing Mape: 0.08681573421103189
chunk_size: 600
Training R²: 0.9404057859175298
Testing R²: 0.8997133288932074
Training Mape: 0.09067458163260457
Testing Mape: 0.1043852857496227
chunk_size: 800
Training R²: 0.9187674118305613
Testing R²: 0.9017633606851183
Training Mape: 0.10736000616186331
Testing Mape: 0.11790905017522024
chunk_size: 1000
Training R²: 0.9058768830937702
Testing R²: 0.8947936657547545
Training Mape: 0.11494967590814045
Testing Mape: 0.12002133393702014


In [12]:
# desktop to InO
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions']
            cyc = row['cpu-cycles']
            ipc = inst/cyc
            features.append([
                row['instructions'],
                row['cpu-cycles'],
                row['br_pred'],
                row['br_mis_pred'],
                row['l1d_cache_rd'],
                row['l1d_cache_wr'],
                row['l1d_cache'],
                row['l1i_cache'],
                row['l1i_cache_refill'],
                row['context-switches'],
                row['l2d_cache_rd'],
                row['l2d_cache_wr'],
                row['l2d_cache'],
                row['l1d_tlb'],
                row['l1d_tlb_refill_rd'],
                row['l1d_tlb_refill_wr'],
                row['dtlb_walk'],
                row['itlb_walk'],
                row['page-faults'],
                row['l2d_tlb_access'],
                row['l2i_tlb_access'],
                row['l1i_tlb_refill'],
                row['iTLB-loads'],
                row['iTLB-load-misses'],
                row['branch-loads'],
                row['dTLB-loads'],
                row['dTLB-load-misses'],
                row['branch-load-misses'],
                row['vfp_spec'],
                row['inst_spec'],
                row['ase_spec'],
                row['bx_stall'],
                row['decode_stall'],
                row['dispatch_stall'],
                row['fx_stall'],
                row['ixa_stall'],
                row['ixb_stall'],
                row['lx_stall'],
                row['sx_stall'],
                row['bus_access'],
                row['mem_access'],
                row['mem_access_rd'],
                row['mem_access_wr'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions:u'] / row['cpu-cycles:u'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_554',
]

chunks = [200, 400, 600, 800, 1000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions',
            'cpu-cycles',
            'br_pred',
            'br_mis_pred',
            'l1d_cache_rd',
            'l1d_cache_wr',
            'l1d_cache',
            'l1i_cache',
            'l1i_cache_refill',
            'context-switches',
            'l2d_cache_rd',
            'l2d_cache_wr',
            'l2d_cache',
            'l1d_tlb',
            'l1d_tlb_refill_rd',
            'l1d_tlb_refill_wr',
            'dtlb_walk',
            'itlb_walk',
            'page-faults',
            'l2d_tlb_access',
            'l2i_tlb_access',
            'l1i_tlb_refill',
            'iTLB-loads',
            'iTLB-load-misses',
            'branch-loads',
            'dTLB-loads',
            'dTLB-load-misses',
            'branch-load-misses',
            'vfp_spec',
            'inst_spec',
            'ase_spec',
            'bx_stall',
            'decode_stall',
            'dispatch_stall',
            'fx_stall',
            'ixa_stall',
            'ixb_stall',
            'lx_stall',
            'sx_stall',
            'bus_access',
            'mem_access',
            'mem_access_rd',
            'mem_access_wr',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 200
Training R²: 0.9850721812785823
Testing R²: 0.9771999035782183
Training Mape: 0.030700735903363457
Testing Mape: 0.036861213203379505
chunk_size: 400
Training R²: 0.9437255548476787
Testing R²: 0.9180059743587625
Training Mape: 0.05313153767254502
Testing Mape: 0.06265859855581324
chunk_size: 600
Training R²: 0.9220933712485649
Testing R²: 0.9044847505295616
Training Mape: 0.06259548139454515
Testing Mape: 0.06901463605179756
chunk_size: 800
Training R²: 0.9046103046005063
Testing R²: 0.8870709503268143
Training Mape: 0.07167440242560176
Testing Mape: 0.07544380794698902
chunk_size: 1000
Training R²: 0.8836445036351613
Testing R²: 0.8705868972646741
Training Mape: 0.07941153606784968
Testing Mape: 0.081622046038825


In [13]:
# server to InO
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions']
            cyc = row['cpu-cycles']
            ipc = inst/cyc
            features.append([
                row['instructions'],
                row['cpu-cycles'], 
                row['br_pred'], 
                row['br_mis_pred'], 
                row['l1d_cache'], 
                row['l1d_tlb'], 
                row['l1d_tlb_rd'], 
                row['l1d_tlb_wr'], 
                row['l2d_cache_rd'], 
                row['l2d_cache_wr'], 
                row['l2d_cache'], 
                row['l1i_cache'], 
                row['l1i_cache_refill'], 
                row['remote_access'], 
                row['dtlb_walk'], 
                row['itlb_walk'], 
                row['l1i_tlb'], 
                row['l2d_tlb'], 
                row['l2d_tlb_rd'], 
                row['l2d_tlb_wr'], 
                row['vfp_spec'], 
                row['inst_spec'], 
                row['ase_spec'], 
                row['stall_backend'], 
                row['stall_frontend'], 
                row['ll_cache_miss_rd'], 
                row['mem_access'], 
                row['mem_access_rd'], 
                row['mem_access_wr'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions:u'] / row['cpu-cycles:u'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_InO_554',
]

chunks = [200, 400, 600, 800, 1000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions',
            'cpu-cycles', 
            'br_pred', 
            'br_mis_pred', 
            'l1d_cache', 
            'l1d_tlb', 
            'l1d_tlb_rd', 
            'l1d_tlb_wr', 
            'l2d_cache_rd', 
            'l2d_cache_wr', 
            'l2d_cache', 
            'l1i_cache', 
            'l1i_cache_refill', 
            'remote_access', 
            'dtlb_walk', 
            'itlb_walk', 
            'l1i_tlb', 
            'l2d_tlb', 
            'l2d_tlb_rd', 
            'l2d_tlb_wr', 
            'vfp_spec', 
            'inst_spec', 
            'ase_spec', 
            'stall_backend', 
            'stall_frontend', 
            'll_cache_miss_rd', 
            'mem_access', 
            'mem_access_rd', 
            'mem_access_wr',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 200
Training R²: 0.9846324229945186
Testing R²: 0.9716242331505356
Training Mape: 0.05363245311145086
Testing Mape: 0.06519395631436878
chunk_size: 400
Training R²: 0.9140350927756694
Testing R²: 0.872693069138293
Training Mape: 0.11304800782822501
Testing Mape: 0.1294043992430083
chunk_size: 600
Training R²: 0.8732449072771586
Testing R²: 0.8222821821067623
Training Mape: 0.1458150216947856
Testing Mape: 0.15415646696458446
chunk_size: 800
Training R²: 0.8409068520899763
Testing R²: 0.8104536388836806
Training Mape: 0.16668525699432774
Testing Mape: 0.17728077571647877
chunk_size: 1000
Training R²: 0.8133683968195505
Testing R²: 0.8076533407234713
Training Mape: 0.18710599826874325
Testing Mape: 0.1884525322820542


In [14]:
# server to OOO
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions']
            cyc = row['cpu-cycles']
            ipc = inst/cyc
            features.append([
                row['instructions'],
                row['cpu-cycles'], 
                row['br_pred'], 
                row['br_mis_pred'], 
                row['l1d_cache'], 
                row['l1d_tlb'], 
                row['l1d_tlb_rd'], 
                row['l1d_tlb_wr'], 
                row['l2d_cache_rd'], 
                row['l2d_cache_wr'], 
                row['l2d_cache'], 
                row['l1i_cache'], 
                row['l1i_cache_refill'], 
                row['remote_access'], 
                row['dtlb_walk'], 
                row['itlb_walk'], 
                row['l1i_tlb'], 
                row['l2d_tlb'], 
                row['l2d_tlb_rd'], 
                row['l2d_tlb_wr'], 
                row['vfp_spec'], 
                row['inst_spec'], 
                row['ase_spec'], 
                row['stall_backend'], 
                row['stall_frontend'], 
                row['ll_cache_miss_rd'], 
                row['mem_access'], 
                row['mem_access_rd'], 
                row['mem_access_wr'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions:u'] / row['cpu-cycles:u'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_500',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_502',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_505',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_520',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_523',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_525',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_531',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_541',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_548',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_557',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_503',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_507',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_508',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_510',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_511',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_519',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_521',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_527',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_538',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_544',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_549',
    '../../Data/traces/inst_aligned_traces/arm_edge_heterogeneous/arm_edge_heterogeneous_OOO_554',
]

chunks = [200, 400, 600, 800, 1000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions',
            'cpu-cycles', 
            'br_pred', 
            'br_mis_pred', 
            'l1d_cache', 
            'l1d_tlb', 
            'l1d_tlb_rd', 
            'l1d_tlb_wr', 
            'l2d_cache_rd', 
            'l2d_cache_wr', 
            'l2d_cache', 
            'l1i_cache', 
            'l1i_cache_refill', 
            'remote_access', 
            'dtlb_walk', 
            'itlb_walk', 
            'l1i_tlb', 
            'l2d_tlb', 
            'l2d_tlb_rd', 
            'l2d_tlb_wr', 
            'vfp_spec', 
            'inst_spec', 
            'ase_spec', 
            'stall_backend', 
            'stall_frontend', 
            'll_cache_miss_rd', 
            'mem_access', 
            'mem_access_rd', 
            'mem_access_wr',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 200
Training R²: 0.9821097239864184
Testing R²: 0.9724075431596473
Training Mape: 0.03160554088913627
Testing Mape: 0.03761502905241759
chunk_size: 400
Training R²: 0.9449098606633984
Testing R²: 0.9186971665200215
Training Mape: 0.052989846653637145
Testing Mape: 0.058905811191312066
chunk_size: 600
Training R²: 0.9186438096305525
Testing R²: 0.8947870489541689
Training Mape: 0.06596054450944439
Testing Mape: 0.07267968969664448
chunk_size: 800
Training R²: 0.8934469567369828
Testing R²: 0.8653106994779889
Training Mape: 0.07613016012493053
Testing Mape: 0.08010674919864448
chunk_size: 1000
Training R²: 0.8590163596564797
Testing R²: 0.8514045976068804
Training Mape: 0.08823413366296554
Testing Mape: 0.09164921837860686


In [15]:
# server to desktop
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

def process_csv_feautures(file_paths, chunk_size):
    all_features = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        features = []
        for index, row in df.iterrows():
            inst = row['instructions']
            cyc = row['cpu-cycles']
            ipc = inst/cyc
            features.append([
                row['instructions'],
                row['cpu-cycles'], 
                row['br_pred'], 
                row['br_mis_pred'], 
                row['l1d_cache'], 
                row['l1d_tlb'], 
                row['l1d_tlb_rd'], 
                row['l1d_tlb_wr'], 
                row['l2d_cache_rd'], 
                row['l2d_cache_wr'], 
                row['l2d_cache'], 
                row['l1i_cache'], 
                row['l1i_cache_refill'], 
                row['remote_access'], 
                row['dtlb_walk'], 
                row['itlb_walk'], 
                row['l1i_tlb'], 
                row['l2d_tlb'], 
                row['l2d_tlb_rd'], 
                row['l2d_tlb_wr'], 
                row['vfp_spec'], 
                row['inst_spec'], 
                row['ase_spec'], 
                row['stall_backend'], 
                row['stall_frontend'], 
                row['ll_cache_miss_rd'], 
                row['mem_access'], 
                row['mem_access_rd'], 
                row['mem_access_wr'],
                ipc])
        all_features.extend(features)
    return all_features


def process_csv_ipc(file_paths, chunk_size):
    all_ipc_values = []
    for file_path in file_paths:
        df = pd.read_csv(file_path + '_' + str(chunk_size))
        df = df.apply(pd.to_numeric, errors='coerce').fillna(0)
        for _, row in df.iterrows():
            all_ipc_values.append(row['instructions'] / row['cpu-cycles'])  
    return all_ipc_values

source = [
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_server/arm_server_1.5GHz_554',
]
    
target = [
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_500',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_502',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_505',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_520',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_523',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_525',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_531',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_541',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_548',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_557',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_503',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_507',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_508',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_510',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_511',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_519',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_521',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_527',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_538',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_544',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_549',
    '../../Data/traces/inst_aligned_traces/arm_desktop/arm_desktop_1.5GHz_554',
]

chunks = [200, 400, 600, 800, 1000]
for chunk_size in chunks:
    in_order_hardware_counters_features = process_csv_feautures(source, chunk_size)
    out_of_order_ipc = process_csv_ipc(target, chunk_size)
    print('chunk_size: ' + str(chunk_size))
    
    data = {
        'In Order': in_order_hardware_counters_features,
        'Out of Order': out_of_order_ipc
    }
    
    length = min(len(in_order_hardware_counters_features), len(out_of_order_ipc))
    
    # Shorten the longen array
    in_order_hardware_counters_features = in_order_hardware_counters_features[:length]
    out_of_order_ipc = out_of_order_ipc[:length]
    
    # separate features and target
    X = pd.DataFrame(
        in_order_hardware_counters_features, 
        columns=[
            'instructions',
            'cpu-cycles', 
            'br_pred', 
            'br_mis_pred', 
            'l1d_cache', 
            'l1d_tlb', 
            'l1d_tlb_rd', 
            'l1d_tlb_wr', 
            'l2d_cache_rd', 
            'l2d_cache_wr', 
            'l2d_cache', 
            'l1i_cache', 
            'l1i_cache_refill', 
            'remote_access', 
            'dtlb_walk', 
            'itlb_walk', 
            'l1i_tlb', 
            'l2d_tlb', 
            'l2d_tlb_rd', 
            'l2d_tlb_wr', 
            'vfp_spec', 
            'inst_spec', 
            'ase_spec', 
            'stall_backend', 
            'stall_frontend', 
            'll_cache_miss_rd', 
            'mem_access', 
            'mem_access_rd', 
            'mem_access_wr',
            'ipc'])  # Features
    y = pd.Series(out_of_order_ipc) 
    
    # split 
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    rf_model = RandomForestRegressor(n_estimators=100)
    gb_model = GradientBoostingRegressor(n_estimators=100)
    ensemble_model = gb_model
    #ensemble_model = VotingRegressor([('rf', rf_model), ('gb', gb_model)])
    ensemble_model.fit(X_train, y_train)
    
    ensemble_predictions = ensemble_model.predict(X_test)
    # Calculate R²
    ensemble_r2 = r2_score(y_test, ensemble_predictions)
    
    #bruh wut
    train_predictions = ensemble_model.predict(X_train)
    test_predictions = ensemble_model.predict(X_test)
    
    # Calculate R² for training and testing sets
    train_r2 = r2_score(y_train, train_predictions)
    train_mape = mean_absolute_percentage_error(y_train, train_predictions)
    test_r2 = r2_score(y_test, test_predictions)
    test_mape = mean_absolute_percentage_error(y_test, test_predictions)
    
    
    print(f"Training R²: {train_r2}")
    print(f"Testing R²: {test_r2}")
    print(f"Training Mape: {train_mape}")
    print(f"Testing Mape: {test_mape}")


chunk_size: 200
Training R²: 0.984458957654995
Testing R²: 0.9764525830628886
Training Mape: 0.032738134031471904
Testing Mape: 0.036490851642208944
chunk_size: 400
Training R²: 0.9170333283322165
Testing R²: 0.8848575430626274
Training Mape: 0.06618370140890283
Testing Mape: 0.07420910235474465
chunk_size: 600
Training R²: 0.8864321110777169
Testing R²: 0.8742467764112702
Training Mape: 0.08234085254966815
Testing Mape: 0.08481815228295908
chunk_size: 800
Training R²: 0.858900908735785
Testing R²: 0.8510251969236655
Training Mape: 0.09835601598953995
Testing Mape: 0.09996188466092307
chunk_size: 1000
Training R²: 0.8283209690205037
Testing R²: 0.8145242536514459
Training Mape: 0.11209781602070532
Testing Mape: 0.11749065940723707
